In [1]:
import pandas as pd

In [42]:
import urllib, urllib.request


In [43]:
from bs4 import BeautifulSoup

In [4]:
import re
def strip(s):
    s=' '.join(s.split())
    return re.sub(r'[^A-Za-z0-9 ]+', '', s)

In [5]:
import fitz
import io
import requests

In [45]:
url="http://www.digitalhumanities.org/dhq/index/title.html"

In [46]:
soup = BeautifulSoup(requests.get(url).text)

C:\Users\lucia\miniconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


'http://creativecommons.org/licenses/by-nd/4.0/'

In [57]:
count=0
links=[]
for para in soup.find_all("a"):
    count+=1
    if count<92:
        continue
    link=para['href']
    if link[0]=="/":
        links.append("http://www.digitalhumanities.org/"+link)
    

In [68]:
soup = BeautifulSoup(requests.get("http://www.digitalhumanities.org/dhq/vol/16/4/000643/000643.html").text)

'2022 16.4'

In [85]:
para.find_all("a")

[<a href="../bios.html#layne-worthey_glen" rel="external">Glen Layne-Worthey
 					</a>,
 <a href="mailto:gworthey_at_illinois_dot_edu" onclick="javascript:window.location.href='mailto:'+deobfuscate('gworthey_at_illinois_dot_edu'); return false;" onkeypress="javascript:window.location.href='mailto:'+deobfuscate('gworthey_at_illinois_dot_edu'); return false;">gworthey_at_illinois_dot_edu</a>,
 <a href="https://orcid.org/0000-0003-2785-0040"><img alt="ORCID logo" height="16" src="https://info.orcid.org/wp-content/uploads/2019/11/orcid_16x16.png" width="16"/>Â https://orcid.org/0000-0003-2785-0040</a>]

'The circus we deserve A front row look at the organization of the annual academic conference for the Digital Humanities'

'Abstract\nAcademic conferences are considered central to the dissemination of research\n\t\t\t\t\tand play a key role in the prestige systems of academia. And yet the\n\t\t\t\t\torganization of these, and the power systems they maintain, have been\n\t\t\t\t\tlittle discussed. What is a conference supposed to achieve? Who and what is\n\t\t\t\t\tit for? The annual Alliance of Digital Humanities Organization (ADHO)â\x80\x99s\n\t\t\t\t\tDigital Humanities conference is a central occasion in the digital\n\t\t\t\t\thumanities academic calendar, and, as an international, interdisciplinary,\n\t\t\t\t\tregular, long-standing, large-scale event, it provides an ideal locus to\n\t\t\t\t\tconsider various aspects of contemporary academic conference organization,\n\t\t\t\t\tand how this impacts the shape and definition of a scholarly field.\n\t\t\t\t\tExamining this annual event allows us to clarify ADHOâ\x80\x99s policies and\n\t\t\t\t\tprocedures to consider how they frame the digital humanities 

In [79]:
count=0
for para in soup.find_all("a"):
    
    print(count)
    count+=1
    print(para)

0
<a class="topnav" href="/dhq/">home</a>
1
<a class="topnav" href="/dhq/submissions/index.html">submissions</a>
2
<a class="topnav" href="/dhq/about/about.html">about dhq</a>
3
<a class="topnav" href="/dhq/people/people.html">dhq people</a>
4
<a class="topnav" href="/dhq/news/news.html">news</a>
5
<a class="topnav" href="/dhq/contact/contact.html">contact</a>
6
<a href="/dhq/vol/16/4/index.html">2022: 16.4</a>
7
<a href="/dhq/preview/index.html">2023: 17.1</a>
8
<a href="/dhq/vol/16/3/index.html">2022: 16.3</a>
9
<a href="/dhq/vol/16/2/index.html">2022: 16.2</a>
10
<a href="/dhq/vol/16/1/index.html">2022: 16.1</a>
11
<a href="/dhq/vol/15/4/index.html">2021: 15.4</a>
12
<a href="/dhq/vol/15/3/index.html">2021: 15.3</a>
13
<a href="/dhq/vol/15/2/index.html">2021: 15.2</a>
14
<a href="/dhq/vol/15/1/index.html">2021: 15.1</a>
15
<a href="/dhq/vol/14/4/index.html">2020: 14.4</a>
16
<a href="/dhq/vol/14/3/index.html">2020: 14.3</a>
17
<a href="/dhq/vol/14/2/index.html">2020: 14.2</a>
18
<a 

In [105]:
para

<a onclick='updateEmulatorMsg();reload("resources/cite_game_demo/01_wolf3d_start.zip", wolfExe)' style="color:green; cursor:pointer;">Wolfenstein 3D didn't get anywhere near.</a>

In [111]:
abstract[9:]

'The past decade has seen a remarkable proliferation of new works of constrained and appropriated writing that prominently incorporate and in turn investigate metadata schemes I argue that these works ought to be of considerable interest not only to critics of contemporary avantgarde writing  but also to media theorists librarians and textual scholars By emphasizing classification protocols conceptual writing makes an implicit case for the interrelationship of these fields Each of the four main books under discussion here  Tan Lins Seven Controlled Vocabularies Craig Dworkins Perverse Library M Nourbese Philips Zong and Simon Morris Getting Inside Jack Kerouacs Head  draws upon preexisting textual archives In doing so these books suggest that processes of data storage classification and transmission are key to how poetry is created recognized and disseminated Conceptual writings attention to information classification protocols offers not only a critique of contemporary models of autho

In [118]:
links

['http://www.digitalhumanities.org//dhq/vol/16/3/000639/000639.html',
 'http://www.digitalhumanities.org//dhq/vol/4/1/000084/000084.html',
 'http://www.digitalhumanities.org//dhq/vol/3/1/000021/000021.html',
 'http://www.digitalhumanities.org//dhq/vol/8/4/000187/000187.html',
 'http://www.digitalhumanities.org//dhq/vol/15/1/000517/000517.html',
 'http://www.digitalhumanities.org//dhq/vol/14/2/000458/000458.html',
 'http://www.digitalhumanities.org//dhq/vol/13/2/000422/000422.html',
 'http://www.digitalhumanities.org//dhq/vol/15/1/000516/000516.html',
 'http://www.digitalhumanities.org//dhq/vol/16/3/000631/000631.html',
 'http://www.digitalhumanities.org//dhq/vol/15/3/000575/000575.html',
 'http://www.digitalhumanities.org//dhq/vol/7/1/000142/000142.html',
 'http://www.digitalhumanities.org//dhq/vol/8/4/000195/000195.html',
 'http://www.digitalhumanities.org//dhq/vol/16/3/000623/000623.html',
 'http://www.digitalhumanities.org//dhq/vol/11/3/000318/000318.html',
 'http://www.digitalhuman

In [115]:
strip(l)

'httpwwwdigitalhumanitiesorgdhqvol162 000605  000605 html'

In [113]:
soup.find_all("a")

[<a href="http://cocoon.apache.org/">Apache Cocoon</a>]

In [117]:
out=[]
for l in links:
    text=""
    soup = BeautifulSoup(requests.get(l).text)
    try:
        date=soup.find_all("a")[66].get_text()
    except:
        continue
    title=strip(soup.find_all("h1")[0].get_text())

    authors=[]
    for para in soup.find_all("a"):
        try:
            if "../bios.html" in para["href"]:
                authors.append(strip(para.text))
        except:
            continue
    for para in soup.find_all("div",  {"class": "ptext"}):
        text+=para.get_text()
        text+=" "
    try:
        abstract=strip(soup.find_all("div", {"id": "abstract"})[0].get_text()[9:])
    except:
        abstract=""
    out.append([date,title, authors,abstract,strip(text), l])

In [119]:
df=pd.DataFrame(out)
df.columns=["date", "title","author", "abstract", "text", "url"]
df.to_csv("dhq.csv")

In [120]:
df

,date,title,author,abstract,text,url
0,2022 16.3,Whole Self to the World Creating Affective Wor...,[Brienne A Adams],This article examines how Black fans utilize s...,In an interview with Harpers Bazaar Issa Rae t...,http://www.digitalhumanities.org//dhq/vol/16/3...
1,2010 4.1,Accessioning the Digital Humanities Report fro...,[Sarah Buchanan],Within the field of archival science recent at...,The Archival Education and Research Institute ...,http://www.digitalhumanities.org//dhq/vol/4/1/...
2,2009 3.1,Acknowledgements and Dedications,"[Gregory Crane, Brent Seales, Melissa Terras]",This collection of essays represent a wide ran...,The essays presented in this collection emerge...,http://www.digitalhumanities.org//dhq/vol/3/1/...
3,2014 8.4,Adobe Photoshop and EighteenthCentury Manuscri...,[Hilary Havens],While research coordinator at the Burney Centr...,Up to this point the primary method for readin...,http://www.digitalhumanities.org//dhq/vol/8/4/...
4,2021 15.1,Advances in Digital Music Iconography Benchmar...,"[Matthia Sabatelli, Nikolay Banar, Marie Cocri...",In this paper we present MINERVA the first ben...,The Digital Humanities constitute an intersect...,http://www.digitalhumanities.org//dhq/vol/15/1...
...,...,...,...,...,...,...
666,2018 12.1,The Writeprints of Man a Stylometric Study of ...,"[Richard Forsyth, David Holmes]",Thomas Paines Rights of Man Part I 1791 Part I...,More than the writings of any other writer tho...,http://www.digitalhumanities.org//dhq/vol/12/1...
667,2016 10.1,A Writing Studies Review of Jim Ridolfo and Wi...,[Kevin G Smith],This is a review of a collection of essays by ...,The twentythree chapters that make up Rhetoric...,http://www.digitalhumanities.org//dhq/vol/10/1...
668,2011 5.3,Writing to be Found and Writing Readers,[John Cayley],Poetic writing for programmable and network me...,In order to begin to write this essay I set ou...,http://www.digitalhumanities.org//dhq/vol/5/3/...
669,2009 3.3,XML Interoperability and the Social Constructi...,[Jerome McDonough],The past decade has seen XML widely adopted wi...,Eleven years after its endorsement by the Worl...,http://www.digitalhumanities.org//dhq/vol/3/3/...


In [6]:
def annotate(i, src):
    date=i["published"]
    title=i["title"]
    authors=i["authors"]
    abstract=strip(i['summary'])
    url = i["links"][-1]["href"]
    tag=i["tags"]
    request = requests.get(url)
    filestream = io.BytesIO(request.content)
    pdf = fitz.open(stream=filestream, filetype="pdf")
    text = ""
    for page in pdf:
        text += page.get_text()
    return [date,title,tag, authors,abstract,strip(text),i["id"], src]
    

In [16]:
def parse_file(path):
    text=""
    doc = fitz.open(path, filetype="pdf")
    for page in doc:
        text += page.get_text()
    return text

In [272]:
def parse(url):
    text = ""
    try:
        request = requests.get(url)
        filestream = io.BytesIO(request.content)
        pdf = fitz.open(stream=filestream, filetype="pdf")
        for page in pdf:
            text += page.get_text()
    except:
        soup = BeautifulSoup(requests.get(url).text)
        [s.extract() for s in soup(['header',"footer",'style', 'script', '[document]', 'head', 'title'])]
        visible_text = soup.getText()
        for para in soup.find_all("p"):
            text+=para.get_text()
            text+=" "
    return text

In [9]:
df=pd.read_csv("open_source.csv")

In [231]:
urls=[]
urls.append(('https://doaj.org/api/search/articles/%22google%20books%22?pageSize=100', "goog"))
urls.append(('https://doaj.org/api/search/articles/%22google%20books%20ngram%20%22?pageSize=100', "goog"))
urls.append(('https://doaj.org/api/search/articles/%22google%20books%20n-gram%20%22?pageSize=100', "goog"))
urls.append(('https://doaj.org/api/search/articles/%22google%20ngram%20%22?pageSize=100', "goog"))
urls.append(('https://doaj.org/api/search/articles/%22google%20n-gram%20%22?pageSize=100', "goog"))

urls.append(('https://doaj.org/api/search/articles/%22hathitrust%22?pageSize=100', "hathi"))
urls.append(('https://doaj.org/api/search/articles/%22hathi%20trust%22?pageSize=100', "hathi"))

urls.append(('https://doaj.org/api/search/articles/%22internet%20archive%22?pageSize=100', "ia"))

urls.append(('https://doaj.org/api/search/articles/%22project%20gutenberg%22?pageSize=100', "guten"))


In [288]:
urls=[]
urls.append(("https://doaj.org/api/search/articles/bibjson.journal.title.exact%3A%22Journal%20of%20Cultural%20Analytics%22?pageSize=100", "jca"))


In [289]:
ret=[]
while urls:
    dat=urls.pop(0)
    url=dat[0]
    mat=dat[1]
    response = requests.get(url)
    if "next" in response.json():
        urls.append((response.json()["next"],mat))
    jr = response.json()["results"]
    for i in jr:
        date=i["created_date"]
        title=i["bibjson"]["title"]
        tag=i["bibjson"]["subject"]
        authors=i["bibjson"]["author"]
        try:
            abstract=i["bibjson"]["abstract"]
        except:
            abstract=""
        journal=i["bibjson"]["journal"]["title"]
        url=i["bibjson"]["link"][0]["url"]
        try:
            text=parse(url)
        except:
            text=""
        ret.append([date,title,tag, authors,abstract,strip(text),url, mat, journal])
    
    
    

In [290]:
df=pd.DataFrame(ret)
df.columns=["date", "title","tag","author", "abstract", "text", "url", "match", "journal"]

In [292]:
df=df.drop_duplicates(subset=["title"])

In [246]:
missing=df[df["text"]==""]

In [ ]:
df["text"][218]=strip("parse("https://programminghistorian.org/en/lessons/text-mining-with-extracted-features")")

In [280]:
df["text"][218]=strip(parse("https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7010436/#B21"))

C:\Users\lucia\AppData\Local\Temp\ipykernel_6684\3372417570.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text"][218]=strip(parse("https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7010436/#B21"))


In [295]:
parse("https://culturalanalytics.org/article/68188-how-network-analysis-uncovers-international-networks-of-smuggling-history-criminals-in-nagasaki-japan-circa-1667")

'Enter the URL below into your favorite RSS reader. Sorry, something went wrong. Please try again.  If this problem reoccurs, please contact Scholastica Support Error message: View more stats This paper takes a network analytic approach to investigating crime in seventeenth-century Japan. In 1667, the Nagasaki magistrate’s office conducted the largest documented smuggling crackdown in Tokugawa Japan (1603–1867), busting a ring of 87 arms traffickers who had been shipping contraband to Chosŏn Korea (1392–1910). I use the office’s “criminal investigation records” (hankachō 犯科帳) to build a dataset of the 94 suspects from ten Japanese towns who were interrogated about their involvement at the time. Using a three-mode network (people, place, crime), the resulting graphs and statistics reveal a new geography of the crime in question: contrary to the conclusions of the original investigators and of modern-day historians who “closely read” their records, the digital analysis relocates the epic

In [294]:
df["t"]

,date,title,tag,author,abstract,text,url,match,journal
0,2020-03-18T16:03:46Z,What’s in a Face? Gender representation of fac...,"[{'code': 'HM401-1281', 'scheme': 'LCC', 'term...","[{'name': 'Ana Jofre'}, {'name': 'Josh Cole'},...","We extracted 327,322 faces from an archive of ...",Journal of Cultural Analytics March 16 2020 Wh...,http://culturalanalytics.scholasticahq.com/art...,jca,Journal of Cultural Analytics
1,2022-03-31T20:09:37Z,Post-War British Women Writers and their Cultu...,"[{'code': 'HM401-1281', 'scheme': 'LCC', 'term...","[{'name': 'Ingo Berensmeyer'}, {'name': 'Sonja...",This article uses a quantitative approach to s...,Journal of Cultural Analytics 71 2022 81107 Jo...,https://culturalanalytics.scholasticahq.com/ar...,jca,Journal of Cultural Analytics
2,2023-01-30T15:45:13Z,Theory-Driven Statistics for the Digital Human...,"[{'code': 'HM401-1281', 'scheme': 'LCC', 'term...",[{'name': 'Ramona Roller'}],The Digital Humanities face the problem of mul...,Enter the URL below into your favorite RSS rea...,https://culturalanalytics.org/article/57764-th...,jca,Journal of Cultural Analytics
3,2020-02-06T16:05:00Z,Topic Modeling the Hàn diăn Ancient Classics (...,"[{'code': 'HM401-1281', 'scheme': 'LCC', 'term...","[{'name': 'Colin Allen'}, {'name': 'Hongliang ...",There is a small but growing literature on lar...,Journalof CulturalAnalytics October 12 2017 To...,http://culturalanalytics.scholasticahq.com/art...,jca,Journal of Cultural Analytics
4,2021-06-17T19:41:17Z,Can We Map Culture?,"[{'code': 'HM401-1281', 'scheme': 'LCC', 'term...","[{'name': 'Ted Underwood'}, {'name': 'Richard ...",Images that convert culture into physical spac...,Journal of Cultural Analytics 6 2021 3251 doi ...,https://culturalanalytics.scholasticahq.com/ar...,jca,Journal of Cultural Analytics
...,...,...,...,...,...,...,...,...,...
84,2019-05-11T12:32:13Z,"The Science of Culture? Social Computing, Digi...","[{'code': 'HM401-1281', 'scheme': 'LCC', 'term...","[{'affiliation': '', 'name': 'Lev Manovich'}]",I developed the concept of cultural analytics ...,For full functionality of this site it is nece...,https://doi.org/10.31235/osf.io/b2y79,jca,Journal of Cultural Analytics
85,2021-12-15T17:27:02Z,"Annotation Guidelines For narrative levels, ti...","[{'code': 'HM401-1281', 'scheme': 'LCC', 'term...",[{'name': 'Edward Kearns'}],These guidelines comprise instructions for the...,Journal of Cultural Analytics December 15 2021...,https://culturalanalytics.scholasticahq.com/ar...,jca,Journal of Cultural Analytics
86,2021-03-30T20:00:03Z,Imaginative Networks: Tracing Connec­tions Amo...,"[{'code': 'HM401-1281', 'scheme': 'LCC', 'term...",[{'name': 'John R. Ladd'}],"This essay uses network metrics (centrality, d...",Journal of Cultural Analytics March 30 2021 Im...,https://culturalanalytics.scholasticahq.com/ar...,jca,Journal of Cultural Analytics
87,2021-05-13T20:39:27Z,Putting the Sorting Hat on J.K. Rowling’s Read...,"[{'code': 'HM401-1281', 'scheme': 'LCC', 'term...","[{'name': 'Wouter Haverals'}, {'name': 'Lindse...",Compared to the large body of research into ge...,Journal of Cultural Analytics May 13 2021 Putt...,https://culturalanalytics.scholasticahq.com/ar...,jca,Journal of Cultural Analytics


In [36]:
df.to_csv("open_source.csv")

In [287]:
df.loc[[151]]["text"]

151    httpsdoiorg1046430phen0058 Great Open Access t...
Name: text, dtype: object

In [152]:
jsonResponse["results"][0]

{'last_updated': '2022-12-21T21:40:01Z',
 'bibjson': {'identifier': [{'id': '1932-6203', 'type': 'eissn'},
   {'id': '10.1371/journal.pone.0212267', 'type': 'doi'}],
  'journal': {'volume': '14',
   'number': '2',
   'country': 'US',
   'issns': ['1932-6203'],
   'publisher': 'Public Library of Science (PLoS)',
   'language': ['EN'],
   'title': 'PLoS ONE'},
  'month': '0',
  'year': '2019',
  'start_page': 'e0212267',
  'subject': [{'code': 'R', 'scheme': 'LCC', 'term': 'Medicine'},
   {'code': 'Q', 'scheme': 'LCC', 'term': 'Science'}],
  'author': [{'name': 'Melissa A Wheeler'},
   {'name': 'Melanie J McGrath'},
   {'name': 'Nick Haslam'}],
  'link': [{'type': 'fulltext',
    'url': 'https://doi.org/10.1371/journal.pone.0212267'}],
  'abstract': 'Trends in the cultural salience of morality across the 20th century in the Anglophone world, as reflected in changing use of moral language, were explored using the Google Books (English language) database. Relative frequencies of 304 moral 

In [177]:
text

'Click through the PLOS taxonomy to find articles in your field. For more information about PLOS Subject Areas, click\n          here.\n         Trends in the cultural salience of morality across the 20th century in the Anglophone world, as reflected in changing use of moral language, were explored using the Google Books (English language) database. Relative frequencies of 304 moral terms, organized into six validated sets corresponding to general morality and the five moral domains proposed by moral foundations theory, were charted for the years 1900 to 2007. Each moral language set displayed unique, often nonlinear historical trajectories. Words conveying general morality (e.g., good, bad, moral, evil), and those representing Purity-based morality, implicating sanctity and contagion, declined steeply in frequency from 1900 to around 1980, when they rebounded sharply. Ingroup-based morality, emphasizing group loyalty, rose steadily over the 20th century. Harm-based morality, focused o

In [153]:
for i in jsonResponse["results"]:
    print(i['bibjson']["link"])

[{'type': 'fulltext', 'url': 'https://doi.org/10.1371/journal.pone.0212267'}]
[{'content_type': 'HTML', 'type': 'fulltext', 'url': 'https://revistas.ucm.es/index.php/ESTR/article/view/53002'}]
[{'content_type': 'html', 'type': 'fulltext', 'url': 'http://www.scielo.br/scielo.php?script=sci_arttext&pid=S1516-05722015000400657&lng=en&tlng=en'}]
[{'type': 'fulltext', 'url': 'http://www.sciencedirect.com/science/article/pii/S2590291120300371'}]
[{'type': 'fulltext', 'url': 'http://journals.openedition.org/anglophonia/3504'}]
[{'content_type': 'html', 'type': 'fulltext', 'url': 'https://jurnal.untag-sby.ac.id/index.php/dih/article/view/5775'}]
[{'type': 'fulltext', 'url': 'http://nzpr.ukma.edu.ua/article/view/153215'}]
[{'content_type': 'HTML', 'type': 'fulltext', 'url': 'http://jlis.it/index.php/jlis/article/view/349'}]
[{'type': 'fulltext', 'url': 'https://isarder.org/2017/vol.9_issue.4_article12_full_text.pdf'}]
[{'type': 'fulltext', 'url': 'http://ried.unizar.es/index.php/revista/article

In [145]:
jsonResponse["next"]

'https://doaj.org/api/v3/search/articles/%22google%20books%22?page=2&pageSize=100'

In [5]:
import feedparser



In [62]:
feed = feedparser.parse('http://export.arxiv.org/api/query?search_query=all:%22internet+archive%22&max_results=10000')


In [87]:
def annotate(i, src):
    date=i["published"]
    title=i["title"]
    authors=i["authors"]
    abstract=strip(i['summary'])
    url = i["links"][-1]["href"]
    tag=i["tags"]
    request = requests.get(url)
    filestream = io.BytesIO(request.content)
    pdf = fitz.open(stream=filestream, filetype="pdf")
    text = ""
    for page in pdf:
        text += page.get_text()
    return [date,title,tag, authors,abstract,strip(text),i["id"], src]
    

In [89]:
ents=[]
feed = feedparser.parse('http://export.arxiv.org/api/query?search_query=all:hathitrust&max_results=10000')
for i in feed.entries:
    ents.append(annotate(i, "hathi"))
feed = feedparser.parse('http://export.arxiv.org/api/query?search_query=all:%22internet+archive%22&max_results=10000')
for i in feed.entries:
    ents.append(annotate(i, "ia"))
feed = feedparser.parse('http://export.arxiv.org/api/query?search_query=all:%22hathi+trust%22&max_results=10000')
for i in feed.entries:
    ents.append(annotate(i, "hathi"))
feed = feedparser.parse('http://export.arxiv.org/api/query?search_query=all:%22google+books%22&max_results=10000')
for i in feed.entries:
    ents.append(annotate(i, "goog"))
feed = feedparser.parse('http://export.arxiv.org/api/query?search_query=all:%22google+ngram%22&max_results=10000')
for i in feed.entries:
    ents.append(annotate(i, "goog"))
feed = feedparser.parse('http://export.arxiv.org/api/query?search_query=all:%22google+books+ngram%22&max_results=10000')
for i in feed.entries:
    ents.append(annotate(i, "goog"))
feed = feedparser.parse('http://export.arxiv.org/api/query?search_query=all:%22project+gutenberg%22&max_results=10000')
for i in feed.entries:
    ents.append(annotate(i, "guten"))
feed = feedparser.parse('http://export.arxiv.org/api/query?search_query=all:%22google+n-gram%22&max_results=10000')
for i in feed.entries:
    ents.append(annotate(i, "goog"))
feed = feedparser.parse('http://export.arxiv.org/api/query?search_query=all:%22google+books+n-gram%22&max_results=10000')
for i in feed.entries:
    ents.append(annotate(i, "goog"))

In [90]:
import pandas as pd

In [91]:
df=pd.DataFrame(ents)

In [133]:
df.to_csv("arxiv.csv")

In [124]:
test=pd.read_csv("arxiv.csv")

In [116]:
df["abstract"]=df["abstract"].apply(strip)

In [117]:
df["text"]=df["text"].apply(strip)

In [120]:
df.replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True, inplace=True)


In [129]:
df["title"].value_counts()

Using Google Ngram Viewer for Scientific Referencing and History of  Science                                                                   4
What Library Digitization Leaves Out: Predicting the Availability of  Digital Surrogates of English Novels                                     3
Freshman or Fresher? Quantifying the Geographic Variation of Internet  Language                                                                2
Both the validity of the cultural tightness index and the association  with creativity and order are spurious -- a comment on Jackson et al    2
Verifying Heaps' law using Google Books Ngram data                                                                                             2
                                                                                                                                              ..
A Framework for Aggregating Private and Public Web Archives                                                                       

In [131]:
df=df.drop_duplicates(subset=["title"])

In [121]:
df["text"][0]

'arXiv190300771v1 csPF 2 Mar 2019 Received 26 April2016 Revised 2 June2016 Accepted 6 June2016 D O Ixxxxxxx ARTICLE TYPE ReliableAccessto M assiveRestricted TextsExperiencebased Evaluation ZongPeng Beth Plale 1SchoolofInform aticsCom putingand EngineeringIndianaUniversityBloom ington IndianaUS Sum m ary Libraries are seeing grow ing num bers ofdigitized textualcorpora thatfrequently com e w ith restrictionson theircontentCom putationalanalysiscorporathatare largew hile ofinterestto scholarscanbecum bersom ebecauseofthecom binationofsizegranularityofaccessandaccess restrictionsEfcientm anagem entofsuchacollectionforgeneralaccessespeciallyunderfailures dependson theprim arystoragesystem In thispaperw eidentifytherequirem entsofm anaging forcom putationalanalysisam assivetextcorpusand useitasbasistoevaluatecandidatestorage solutionsThestudybased on the59 billion pagecollection oftheH athiTrustdigitallibraryO ur ndingsled to the choice ofCassandra 3x forthe prim ary back end storew hich is

In [27]:
import io
import requests
from PyPDF import PdfReader
headers = {'User-Agent': 'Mozilla/5.0 (X11; Windows; Windows x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.114 Safari/537.36'}

url = 'http://arxiv.org/pdf/1903.00771v1'
response = requests.get(url=url, headers=headers, timeout=120)
on_fly_mem_obj = io.BytesIO(response.content)
pdf_file = PdfReader(on_fly_mem_obj)

ImportError: cannot import name 'PdfReader' from 'PyPDF4' (C:\Users\lucia\miniconda3\lib\site-packages\PyPDF4\__init__.py)

In [26]:
#importing required modules
import PyPDF4

In [16]:
pdf_file

In [36]:

url = "http://arxiv.org/pdf/2301.10781v1"
request = requests.get(url)
filestream = io.BytesIO(request.content)
pdf = fitz.open(stream=filestream, filetype="pdf")

In [37]:
import fitz # install using: pip install PyMuPDF

text = ""
for page in pdf:
    text += page.get_text()

print(text)

Springer Nature 2021 LATEX template
Generalizability in Document Layout Analysis for Scientiﬁc
Article Figure & Caption Extraction
Jill Naiman1,2*
1*School of Information Sciences, University of Illinois, Urbana-Champaign, Daniel ST,
Champaign, 61820, Illinois, USA.
2National Center for Supercomputing Applications, University of Illinois,
Urbana-Champaign, W Clark ST, Urbana, 61801, Illinois, USA.
Corresponding author(s). E-mail(s): jnaiman@illinois.edu;
Abstract
The lack of generalizability – in which a model trained on one dataset cannot provide accurate results
for a diﬀerent dataset – is a known problem in the ﬁeld of document layout analysis. Thus, when
a model is used to locate important page objects in scientiﬁc literature such as ﬁgures, tables, cap-
tions, and math formulas, the model often cannot be applied successfully to new domains. While
several solutions have been proposed, including newer and updated deep learning models, larger hand-
annotated datasets, and the generat

In [24]:
page.extract_text()

'AUTHORONE ET AL 13\nCassandraDataM odelDetai l s\nForCassandra,HTRC datai sstoredi nCassandracol umnfami l yi nthefol l owi ngdatamodelforbothvol umel evelrandom acc essandpagel evel \nrandom access. \n1.Vol umeI D i susedasparti ti onkey. \n2.pagesequencei susedascl usteri ngkey. \n3.vol umel evelmetadata( e. g.vol umesi ze)i sstoredi nstat i ccol umns. \n4.pagetextualcontentsarestoredi ndynami ccol umn"conte nts" . \nTobemorespeci ﬁc,theCQLstatementtocreateaCassandraCo l umnfami l yusedbyexperi mentsi npapercanbewri ttenasbel ow: \nString createTableStr="CREATE TABLE " + col umnFami l yName + " ( "\n+ "volumeID text, "\n+ "accessLevel int static , "\n+ "language text static , "\n+ "volumeByteCount bigint static , "\n+ "volum eCharacterCount int static , "\n+ "sequence text, "\n+ "byteCount bigint, "\n+ "characterCount int, "\n+ "contents text, "\n+ "checksum text, "\n+ "checksumType text, "\n+ "pageNumberLabel text, "\n+ "PRI MARY KEY (vol umeID , sequence))"; \nForvol umel evelrand

In [17]:
text = ""
for page in pdf_file.pages:
    text += page.extract_text() + "\n"

In [19]:
from tika import parser

In [20]:
raw = parser.from_file(on_fly_mem_obj)


AttributeError: '_io.BytesIO' object has no attribute 'name'

In [18]:
text

'arXiv:1903.00771v1  [cs.PF]  2 Mar 2019Recei ved26Apri l2016; Revi sed2June2016; Accepted6June2016\nDOI:xxx/xxxx\nARTICLETYPE\nRel i abl eAccesstoM assi veRestri ctedTexts:Experi ence-b ased\nEval uati on\nZongPeng* |BethPl al e\n1SchoolofInformati cs,Computi ng,and\nEngi neeri ng,Indi anaUni versi tyBl oomi ngton, \nIndi ana,USSummary\nLi brari esareseei nggrowi ngnumbersofdi gi ti zed textualc orporathatfrequentl ycomewi th\nrestri cti onsonthei rcontent.Computati onalanal ysi scor porathatarel arge,whi l eofi nterestto\nschol ars,canbecumbersomebecauseofthecombi nati onofsi ze,granul ari tyofaccess,andaccess\nrestri cti ons.Efﬁci entmanagementofsuchacol l ecti onfor generalaccessespeci al l yunderfai l ures\ndependsonthepri marystoragesystem.Inthi spaper,wei den ti fytherequi rementsofmanagi ng\nforcomputati onalanal ysi samassi vetextcorpusandusei ta sbasi stoeval uatecandi datestorage\nsol uti ons.Thestudybasedonthe5. 9bi l l i onpagecol l ecti o noftheHathi Trustdi gi tall i bra